In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# import matplotlib.pyplot as plt
# from joblib import Parallel, delayed
# from sklearn.ensemble import RandomForestRegressor

2023-04-01 16:58:06.548372: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-01 16:58:06.550311: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-01 16:58:06.590966: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-01 16:58:06.591586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 16:58:07.289188: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
strs_ref = pd.read_csv('./mesh_18-143.csv')
# strs_ref


In [5]:
x = pd.read_csv('./mesh_18-143_cord.csv')
x = x.loc[:, ~x.columns.str.contains('^Unnamed')]
print(x.head());


               x          y
0       0.000000   0.000000
1       2.666667   2.444444
2       5.333333   4.888889
3       8.000000   7.333333
4      10.666667   9.777778
...          ...        ...
65918  46.657343  59.552448
65919  46.993007  59.664336
65920  47.328671  59.776224
65921  47.664336  59.888112
65922  48.000000  60.000000

[65923 rows x 2 columns]
           x         y
0   0.000000  0.000000
1   2.666667  2.444444
2   5.333333  4.888889
3   8.000000  7.333333
4  10.666667  9.777778


In [7]:
y = strs_ref
y = y.loc[:, ~y.columns.str.contains('^Unnamed')]
y = y.loc[:, ~y.columns.str.contains('^x')]
y = y.loc[:, ~y.columns.str.contains('^y')]
y.head()


,sigma_x,sigma_y,sigma_xy
0,0.038969,0.012990,0.031987
1,0.042593,0.038722,0.036876
2,0.052135,0.033737,0.048589
3,0.059190,0.052859,0.055300
4,0.073768,0.045675,0.067387


In [8]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []

    nodes_increment = (last_layer_nodes - first_layer_nodes) / (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    return layers


In [9]:
import time
start = time.time()


In [10]:
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    #newmodel
    model1 = Sequential()
    n_nodes = FindLayerNodesLinear(
        n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i == 1:
            model1.add(Dense(first_layer_nodes, activation=activation_func))
        else:
            model1.add(Dense(n_nodes[i-1], activation=activation_func))

    model1.add(Dense(units=3, activation='linear'))
    model1.compile(
        loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
    return model1


seed = 7
np.random.seed(seed)


In [12]:
X = x
Y = y
# print(X)
# print(Y)


               x          y
0       0.000000   0.000000
1       2.666667   2.444444
2       5.333333   4.888889
3       8.000000   7.333333
4      10.666667   9.777778
...          ...        ...
65918  46.657343  59.552448
65919  46.993007  59.664336
65920  47.328671  59.776224
65921  47.664336  59.888112
65922  48.000000  60.000000

[65923 rows x 2 columns]
        sigma_x   sigma_y  sigma_xy
0      0.038969  0.012990  0.031987
1      0.042593  0.038722  0.036876
2      0.052135  0.033737  0.048589
3      0.059190  0.052859  0.055300
4      0.073768  0.045675  0.067387
...         ...       ...       ...
65918 -0.017590  0.112467 -0.093654
65919 -0.203363 -0.154760  0.040485
65920 -0.002240  0.142478 -0.103031
65921 -0.266350 -0.214639  0.093489
65922  0.010705  0.207424 -0.140326

[65923 rows x 3 columns]


In [ ]:

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [256, 512, 1024, 2048]
epochs = [50, 100, 150, 200, 300]
n_layers = [5, 8, 10]
first_layer_nodes = [50, 60, 80]
last_layer_nodes = [5, 10]
activation_funcs = ['relu', 'sigmoid', 'tanh']
param_grid = dict(epochs=epochs, batch_size=batch_size, n_layers=n_layers,
                  first_layer_nodes=first_layer_nodes, last_layer_nodes=last_layer_nodes, activation_func=activation_funcs)
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    n_jobs=-1, cv=5, verbose=2)
#print(1, neurons1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
end = time.time()
print("The time of execution of above program is :", end-start)
